# **Stage B Project:** Lightgbm_PP22_T615

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## **1. Load Data**



In [ ]:
from google.colab import files
files.upload()

In [ ]:
!pip install kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download zanjibar/japan-trade-statistics
!unzip /content/japan-trade-statistics -d /content/kaggle_input

In [7]:
import sqlite3
import pandas as pd
from pandas import read_sql_query, read_sql_table

In [42]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [41]:
def read_sqlite(dbfile):
  with sqlite3.connect(dbfile) as dbcon:
    tables = list(read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", dbcon)['name'])
    out = {tbl : read_sql_query(f"SELECT * from {tbl}", dbcon) for tbl in tables}
  return out

### **a. Codes.db**

In [9]:
codes = read_sqlite("kaggle_input/codes.db")
print(codes.keys())

dict_keys(['country_jpn', 'hs2_jpn', 'hs2_eng', 'hs4_eng', 'hs6_jpn', 'hs6_eng', 'hs9_jpn', 'hs9_eng', 'country_eng', 'hs4_jpn', 'custom_tmp', 'custom'])


In [11]:
df_codes_country = codes['country_eng']
df_codes_country.head(10)

Country Country_name  Area
0     103        Korea  Asia
1     104  North_Korea  Asia
2     105        China  Asia
3     106       Taiwan  Asia
4     107     Mongolia  Asia
5     108    Hong_Kong  Asia
6     110     Viet_Nam  Asia
7     111     Thailand  Asia
8     112    Singapore  Asia
9     113     Malaysia  Asia

In [12]:
print("Null Values:\n\n", df_codes_country.isnull().sum(), "\n")
print("Duplicated Values: ", df_codes_country.duplicated().any())

Null Values:

 Country         0
Country_name    0
Area            0
dtype: int64 

Duplicated Values:  False


In [13]:
print("Number of rows: ", len(df_codes_country))

Number of rows:  232


In [14]:
code_to_name = dict(zip((df_codes_country['Country']),df_codes_country['Country_name']))
code_to_name = {int(k):v for k,v in code_to_name.items()}
print(code_to_name)

{103: 'Korea', 104: 'North_Korea', 105: 'China', 106: 'Taiwan', 107: 'Mongolia', 108: 'Hong_Kong', 110: 'Viet_Nam', 111: 'Thailand', 112: 'Singapore', 113: 'Malaysia', 116: 'Brunei', 117: 'Philippines', 118: 'Indonesia', 120: 'Cambodia', 121: 'Lao', 122: 'Myanmar', 123: 'India', 124: 'Pakistan', 125: 'Sri_Lanka', 126: 'Maldives', 127: 'Bangladesh', 128: 'Timor-Leste', 129: 'Macao', 130: 'Afghanistan', 131: 'Nepal', 132: 'Bhutan', 133: 'Iran', 134: 'Iraq', 135: 'Bahrain', 137: 'Saudi_Arabia', 138: 'Kuwait', 140: 'Qatar', 141: 'Oman', 143: 'Israel', 144: 'Jordan', 145: 'Syria', 146: 'Lebanon', 147: 'United_Arab_Emirates', 149: 'Yemen', 150: 'Azerbaijan', 151: 'Armenia', 152: 'Uzbekistan', 153: 'Kazakhstan', 154: 'Kyrgyz', 155: 'Tajikistan', 156: 'Turkmenistan', 157: 'Georgia', 158: 'West_Bank_and_Gaza_Strip', 201: 'Iceland', 202: 'Norway', 203: 'Sweden', 204: 'Denmark', 205: 'United_Kingdom', 206: 'Ireland', 207: 'Netherlands', 208: 'Belgium', 209: 'Luxembourg', 210: 'France', 211: 'Mona

### **b. ym_2018.db**

In [15]:
ym_2018 = read_sqlite("kaggle_input/ym_2018.db")
print(ym_2018)

{'ym_2018':          exp_imp  Year month  Country Unit1  ...  Value  hs2   hs4     hs6        hs9
0              1  2018    01      205        ...      0   01  0101  010121  010121000
1              1  2018    01      206        ...      0   01  0101  010121  010121000
2              1  2018    01      213        ...  30014   01  0101  010121  010121000
3              1  2018    01      220        ...   9020   01  0101  010121  010121000
4              1  2018    01      304        ...   1500   01  0101  010121  010121000
...          ...   ...   ...      ...   ...  ...    ...  ...   ...     ...        ...
2824051        2  2018    12      619        ...      0   00  0000  000000  000000099
2824052        2  2018    12      620        ...  20444   00  0000  000000  000000099
2824053        2  2018    12      626        ...      0   00  0000  000000  000000099
2824054        2  2018    12      628        ...      0   00  0000  000000  000000099
2824055        2  2018    12      702     

In [16]:
df_2018 = ym_2018['ym_2018']
df_2018.head()

exp_imp  Year month  Country Unit1  ...  Value  hs2   hs4     hs6        hs9
0        1  2018    01      205        ...      0   01  0101  010121  010121000
1        1  2018    01      206        ...      0   01  0101  010121  010121000
2        1  2018    01      213        ...  30014   01  0101  010121  010121000
3        1  2018    01      220        ...   9020   01  0101  010121  010121000
4        1  2018    01      304        ...   1500   01  0101  010121  010121000

[5 rows x 13 columns]

In [17]:
print("Null Values:\n\n", df_2018.isnull().sum(), "\n")
print("Duplicated Values: ", df_2018.duplicated().any())

Null Values:

 exp_imp    0
Year       0
month      0
Country    0
Unit1      0
Unit2      0
Q1         0
Q2         0
Value      0
hs2        0
hs4        0
hs6        0
hs9        0
dtype: int64 

Duplicated Values:  False


In [18]:
print("Number of rows: ", len(df_2018))

Number of rows:  2824056


### **c. custom_2020.db**

In [19]:
custom_2020 = read_sqlite("kaggle_input/custom_2020.db")
print(custom_2020)

{'custom_2020':          exp_imp  Year month  Country Unit1  ... hs2   hs4     hs6        hs9 Custom
0              1  2020    01      103        ...  00  0000  000000  000000190    100
1              1  2020    01      105        ...  00  0000  000000  000000190    100
2              1  2020    01      106        ...  00  0000  000000  000000190    100
3              1  2020    01      107        ...  00  0000  000000  000000190    100
4              1  2020    01      108        ...  00  0000  000000  000000190    100
...          ...   ...   ...      ...   ...  ...  ..   ...     ...        ...    ...
4494107        2  2020    07      111    NO  ...  95  9506  950699  950699000    907
4494108        2  2020    07      304    NO  ...  95  9506  950699  950699000    907
4494109        2  2020    07      305    NO  ...  95  9506  950699  950699000    907
4494110        2  2020    07      105        ...  96  9618  961800  961800000    907
4494111        2  2020    07      218        ... 

In [20]:
df_custom_2020 = custom_2020['custom_2020']
df_custom_2020.head()

exp_imp  Year month  Country Unit1  ... hs2   hs4     hs6        hs9 Custom
0        1  2020    01      103        ...  00  0000  000000  000000190    100
1        1  2020    01      105        ...  00  0000  000000  000000190    100
2        1  2020    01      106        ...  00  0000  000000  000000190    100
3        1  2020    01      107        ...  00  0000  000000  000000190    100
4        1  2020    01      108        ...  00  0000  000000  000000190    100

[5 rows x 14 columns]

In [21]:
print("Number of rows: ", len(df_custom_2020))

Number of rows:  4494112


In [22]:
print("Null Values:\n\n", df_custom_2020.isnull().sum(), "\n")
print("Duplicated Values: ", df_custom_2020.duplicated().any())

Null Values:

 exp_imp    0
Year       0
month      0
Country    0
Unit1      0
Unit2      0
Q1         0
Q2         0
Value      0
hs2        0
hs4        0
hs6        0
hs9        0
Custom     0
dtype: int64 

Duplicated Values:  False


In [23]:
unknown_countries = [country for country in df_custom_2020['Country'].unique() if country not in code_to_name]
print(unknown_countries)

[338, 250, 249]


In [24]:
df_custom_2020 = df_custom_2020[ df_custom_2020['Country'].isin(unknown_countries)==False ]

In [25]:
df_custom_2020['Country'].replace(code_to_name, inplace=True)
df_custom_2020.head()

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:6619: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


exp_imp  Year month    Country Unit1  ... hs2   hs4     hs6        hs9 Custom
0        1  2020    01      Korea        ...  00  0000  000000  000000190    100
1        1  2020    01      China        ...  00  0000  000000  000000190    100
2        1  2020    01     Taiwan        ...  00  0000  000000  000000190    100
3        1  2020    01   Mongolia        ...  00  0000  000000  000000190    100
4        1  2020    01  Hong_Kong        ...  00  0000  000000  000000190    100

[5 rows x 14 columns]

In [26]:
pd.DataFrame( df_custom_2020.groupby(['month','Country']).sum() ).head()

exp_imp    Year      Q1        Q2    Value
month Country                                                     
01    Afghanistan           152  288860  178300    266953   376338
      Albania               326  369660   23601    117004   152643
      Algeria               352  684780   35719  23745051  3294847
      American_Oceania       18   18180      88        59    10480
      American_Samoa         23   42420       0        73    11314

## **[custom_2021)]**

In [49]:
custom_2021 = read_sqlite("kaggle_input/custom_2021.db")
custom_2021

{'custom_2021':         index  exp_imp  Year month  ...        hs9   Q1       Q2    Value
 0           0        1  2021    01  ...  000000190    0   583458  1247328
 1           1        1  2021    01  ...  000000190    0  1302234  4849164
 2           2        1  2021    01  ...  000000190    0   477223  1343050
 3           3        1  2021    01  ...  000000190    0     2764     3565
 4           4        1  2021    01  ...  000000190    0   528471  9963309
 ...       ...      ...   ...   ...  ...        ...  ...      ...      ...
 300370    903        2  2021    01  ...  940171090   40      466     2276
 300371    904        2  2021    01  ...  940370000   20      467     1063
 300372    905        2  2021    01  ...  940510010    0      547     3321
 300373    906        2  2021    01  ...  940599000    0       81      708
 300374    907        2  2021    01  ...  950440000  947      277     1114
 
 [300375 rows x 14 columns]}

In [50]:
df_custom_2021 = custom_2021['custom_2021']
df_custom_2021.head()

index  exp_imp  Year month      ym  ...     hs6        hs9 Q1       Q2    Value
0      0        1  2021    01  202101  ...  000000  000000190  0   583458  1247328
1      1        1  2021    01  202101  ...  000000  000000190  0  1302234  4849164
2      2        1  2021    01  202101  ...  000000  000000190  0   477223  1343050
3      3        1  2021    01  202101  ...  000000  000000190  0     2764     3565
4      4        1  2021    01  202101  ...  000000  000000190  0   528471  9963309

[5 rows x 14 columns]

In [53]:
df_custom_2021.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300375 entries, 0 to 300374
Data columns (total 14 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   index    300375 non-null  int64 
 1   exp_imp  300375 non-null  int64 
 2   Year     300375 non-null  int64 
 3   month    300375 non-null  object
 4   ym       300375 non-null  int64 
 5   Country  300375 non-null  int64 
 6   Custom   300375 non-null  int64 
 7   hs2      300375 non-null  object
 8   hs4      300375 non-null  object
 9   hs6      300375 non-null  object
 10  hs9      300375 non-null  object
 11  Q1       300375 non-null  int64 
 12  Q2       300375 non-null  int64 
 13  Value    300375 non-null  int64 
dtypes: int64(9), object(5)
memory usage: 32.1+ MB


In [54]:
df_custom_2021.isnull().sum()

index      0
exp_imp    0
Year       0
month      0
ym         0
Country    0
Custom     0
hs2        0
hs4        0
hs6        0
hs9        0
Q1         0
Q2         0
Value      0
dtype: int64

In [55]:
df_custom_2021.duplicated().any()

False

In [56]:
unknown_countries = [country for country in df_custom_2021['Country'].unique() if country not in code_to_name]
print(unknown_countries)

[338, 249]


In [57]:
df_custom_2021 = df_custom_2021[ df_custom_2021['Country'].isin(unknown_countries)==False ]

In [58]:
df_custom_2021['Country'].replace(code_to_name, inplace=True)
df_custom_2021.head()

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:6619: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


index  exp_imp  Year month      ym  ...     hs6        hs9 Q1       Q2    Value
0      0        1  2021    01  202101  ...  000000  000000190  0   583458  1247328
1      1        1  2021    01  202101  ...  000000  000000190  0  1302234  4849164
2      2        1  2021    01  202101  ...  000000  000000190  0   477223  1343050
3      3        1  2021    01  202101  ...  000000  000000190  0     2764     3565
4      4        1  2021    01  202101  ...  000000  000000190  0   528471  9963309

[5 rows x 14 columns]

In [64]:
pd.DataFrame( df_custom_2021.groupby(['month','Country']).sum() ).head()

index  exp_imp    Year  ...     Q1        Q2    Value
month Country                                    ...                          
01    Afghanistan       426089       50   90945  ...  21829    246363   283019
      Albania           340112      106  119239  ...   8729     97068   120858
      Algeria           765546       90  175827  ...  94409  68031511  3908457
      American_Oceania    9283        2    2021  ...      0         0     1028
      Andorra            22727        2    2021  ...      0         9     1236

[5 rows x 8 columns]